# Autolaparo
It can be seen that
- Left / Right, zoom-in / zoom-out motions correlate a little with camera motion extraction
- Individual sequences contain multiple labels and are not well isolated
- Impossible to use for evaluation
- Should poll motion sequences from heichole / cholec80 instead, using mean or similar

- for out purposes, one should learn on **non-static data**, ie data where running average exceeds certain threshold

In [ ]:
import sys
sys.path.append("../..")
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

from utils.io import load_yaml

server = "local"
server = load_yaml("../../config/servers.yml")[server]

increment = 5

database = server["database"]["location"]
prefix = "autolaparo_single_frames/AutoLaparo_Task2"
motion_pickle = f"23_02_24_pre_processed_frame_increment_{increment}_frames_between_clips_1_log.pkl"
label_pickle = "laparoscope_motion_label.csv"

# from readme
motion_dict = {
    0: "Static",
    1: "Up",
    2: "Down",
    3: "Left",
    4: "Right",
    5: "Zoom-in",
    6: "Zoom-out",
}

motion_df = pd.read_pickle(os.path.join(database, prefix, motion_pickle))
motion_df = motion_df.dropna().reset_index(drop=True)

# motion_df = motion_df.sort_values(by=["vid", "frame"]).reset_index(drop=True)
label_df = pd.read_csv(os.path.join(database, prefix, label_pickle))
label_df["Clip"] = label_df["Clip"].apply(lambda x: x - 1)

print(label_df)
print(motion_df)

In [ ]:
from utils.processing import classify_duv_motion, dataframe_duv_running_average

# de-couple duv

df_split, mean, std = dataframe_duv_running_average(motion_df, window=20)

df_split["classes"] = df_split.apply(lambda x: classify_duv_motion(x["duv_0_0"], x["duv_0_1"], x["duv_1_0"], x["duv_1_1"], x["duv_2_0"], x["duv_2_1"], x["duv_3_0"], x["duv_3_1"], mean + 1.), axis=1)
print(df_split)
print(df_split.classes.unique())
# print(df_split[df_split["classes"] == "left"])

# rolling average per video
# 25 fps at 5 frame increment
# df_split.groupby("vid")
id = 60
print("autolaparo class: {}".format(motion_dict[label_df.iloc[id]["Label"]]))
df_split_id = df_split[df_split["vid"] == id]
df_split_id = df_split_id[::increment]
# df_split_id = df_split_id.rolling(20, min_periods=1).mean()
df_split_id.duv_0_0.plot()
df_split_id.duv_0_1.plot()
df_split_id.duv_1_0.plot()
df_split_id.duv_1_1.plot()
df_split_id.duv_2_0.plot()
df_split_id.duv_2_1.plot()
df_split_id.duv_3_0.plot()
df_split_id.duv_3_1.plot()


# Analyze Motion

In [ ]:
motion_df = motion_df[::increment]
print(motion_df.shape)

for key in motion_dict:
    print(f"Analyzing {motion_dict[key]} motion")
    clips = label_df[label_df["Label"] == key].Clip.values
    frames = motion_df[motion_df["vid"].isin(clips)].reset_index(drop=True)
    
    duv0 = np.array(frames["duv"].apply(lambda x: x[0]).to_list())
    duv1 = np.array(frames["duv"].apply(lambda x: x[1]).to_list())
    duv2 = np.array(frames["duv"].apply(lambda x: x[2]).to_list())
    duv3 = np.array(frames["duv"].apply(lambda x: x[3]).to_list())

    print(f"duv0: {duv0.mean(axis=0)} pm {duv0.std(axis=0)}")
    print(f"duv1: {duv1.mean(axis=0)} pm {duv1.std(axis=0)}")
    print(f"duv2: {duv2.mean(axis=0)} pm {duv2.std(axis=0)}")
    print(f"duv3: {duv3.mean(axis=0)} pm {duv3.std(axis=0)}")

    plt.scatter(duv0[:, 0], duv0[:, 1], label="duv0")
    plt.scatter(duv1[:, 0], duv1[:, 1], label="duv1")
    plt.scatter(duv2[:, 0], duv2[:, 1], label="duv2")
    plt.scatter(duv3[:, 0], duv3[:, 1], label="duv3")
    plt.xlim(-100, 100)
    plt.ylim(-100, 100)
    plt.legend()
    plt.show()

# Motion as Sequence

In [ ]:
horizon = 2

def scalar_product(a: np.ndarray) -> np.ndarray:
    return np.array([np.dot(a[i], a[i+1]) / (np.linalg.norm(a[i]) * np.linalg.norm(a[i+1])) for i in range(len(a) - 1)])

def no_sign_change(a: np.ndarray, horizon: int=10) -> np.ndarray:
    no_change = [True]*(len(a)-horizon)
    for i in range(len(a) - horizon):
        for j in range(horizon):
            no_change[i] = no_change[i] and np.sign(a[i]) == np.sign(a[i+j])
    return np.array(no_change)
        

def no_sign_changes(a: np.ndarray, b: np.ndarray, c: np.ndarray, d: np.ndarray) -> np.ndarray:
    return np.array([a[i] and b[i] and c[i] and d[i] for i in range(len(a))])

for key in motion_dict:
    print(f"Analyzing {motion_dict[key]} motion")
    clips = label_df[label_df["Label"] == key].Clip.values
    frames = motion_df[motion_df["vid"].isin(clips)].reset_index(drop=True)
    
    duv0 = np.array(frames["duv"].apply(lambda x: x[0]).to_list())
    duv1 = np.array(frames["duv"].apply(lambda x: x[1]).to_list())
    duv2 = np.array(frames["duv"].apply(lambda x: x[2]).to_list())
    duv3 = np.array(frames["duv"].apply(lambda x: x[3]).to_list())

    # compute scalar product of previous and current motion vectors
    duv0_no_sign_change = no_sign_change(scalar_product(duv0), horizon)
    duv1_no_sign_change = no_sign_change(scalar_product(duv1), horizon)
    duv2_no_sign_change = no_sign_change(scalar_product(duv2), horizon)
    duv3_no_sign_change = no_sign_change(scalar_product(duv3), horizon)

    no_changes = no_sign_changes(duv0_no_sign_change, duv1_no_sign_change, duv2_no_sign_change, duv3_no_sign_change)

    # get duvs where no sign change occured
    duv0 = duv0[:-horizon, :][np.argwhere(no_changes)].squeeze()
    duv1 = duv1[:-horizon, :][np.argwhere(no_changes)].squeeze()
    duv2 = duv2[:-horizon, :][np.argwhere(no_changes)].squeeze()
    duv3 = duv3[:-horizon, :][np.argwhere(no_changes)].squeeze()

    plt.scatter(duv0[:, 0], duv0[:, 1], label="duv0")
    plt.scatter(duv1[:, 0], duv1[:, 1], label="duv1")
    plt.scatter(duv2[:, 0], duv2[:, 1], label="duv2")
    plt.scatter(duv3[:, 0], duv3[:, 1], label="duv3")
    plt.xlim(-100, 100)
    plt.ylim(-100, 100)
    plt.legend()
    plt.show()

    
    


In [ ]:
for key in motion_dict:
    print(f"Analyzing {motion_dict[key]} motion")
    clips = label_df[label_df["Label"] == key].Clip.values
    frames = motion_df[motion_df["vid"].isin(clips)].reset_index(drop=True)
    
    duv0 = np.array(frames["duv"].apply(lambda x: x[0]).to_list())
    duv1 = np.array(frames["duv"].apply(lambda x: x[1]).to_list())
    duv2 = np.array(frames["duv"].apply(lambda x: x[2]).to_list())
    duv3 = np.array(frames["duv"].apply(lambda x: x[3]).to_list())

    # duv0 = duv0[np.linalg.norm(duv0, axis=1) > 40]
    # duv1 = duv1[np.linalg.norm(duv1, axis=1) > 40]
    # duv2 = duv2[np.linalg.norm(duv2, axis=1) > 40]
    # duv3 = duv3[np.linalg.norm(duv3, axis=1) > 40]

    duv0 = duv0.sum(axis=0)
    duv1 = duv1.sum(axis=0)
    duv2 = duv2.sum(axis=0)
    duv3 = duv3.sum(axis=0)

    max_norm = 0
    norms = [
        np.linalg.norm(duv0),
        np.linalg.norm(duv1),
        np.linalg.norm(duv2),
        np.linalg.norm(duv3),
    ]
    for norm in norms:
        if norm > max_norm:
            max_norm = norm

    duv0 = duv0 / max_norm
    duv1 = duv1 / max_norm
    duv2 = duv2 / max_norm
    duv3 = duv3 / max_norm

    plt.scatter(duv0[1], duv0[0], label="duv0")
    plt.scatter(duv1[1], duv1[0], label="duv1")
    plt.scatter(duv2[1], duv2[0], label="duv2")
    plt.scatter(duv3[1], duv3[0], label="duv3")
    plt.xlim(-2, 2)
    plt.ylim(-2, 2)
    plt.legend()
    plt.show()


# Generate Video of classified Videos

In [ ]:
import torch
import numpy as np
from kornia import tensor_to_image, image_to_tensor
from kornia.geometry import resize, warp_perspective
import PIL
from IPython.display import Image, display, clear_output, TextDisplayObject

from utils import four_point_homography_to_matrix, image_edges
from utils.viz import create_blend_from_four_point_homography

shape = [240, 320]
display_shape = [480, 640]
motion_id = 3

# motion_dict = {
#     0: "Static",
#     1: "Up",
#     2: "Down",
#     3: "Left",
#     4: "Right",
#     5: "Zoom-in",
#     6: "Zoom-out",
# }

# unique_motion_df = motion_df[motion_df["vid"].isin(label_df[label_df["Label"] == motion_id].Clip.values)].reset_index(drop=True)
unique_motion_df = df_split[df_split["vid"].isin(label_df[label_df["Label"] == motion_id].Clip.values)].reset_index(drop=True)

# write a video using opencv
import cv2
import os

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(f'/tmp/output_{motion_id}.avi',fourcc, 10.0, (1280, 480))

cnt = 0
for idx, row in unique_motion_df.iterrows():
    img = np.load(os.path.join(database, prefix, row.folder, row.file))
    wrp = np.load(os.path.join(database, prefix, unique_motion_df.iloc[idx + increment].folder, unique_motion_df.iloc[idx + increment].file))
    img, wrp = resize(image_to_tensor(img, keepdim=False).float()/255., shape), resize(image_to_tensor(wrp, keepdim=False).float()/255., shape)

    # duv = torch.tensor(row.duv).unsqueeze(0)
    duv = torch.tensor([
        [row.duv_0_0, row.duv_0_1],
        [row.duv_1_0, row.duv_1_1],
        [row.duv_2_0, row.duv_2_1],
        [row.duv_3_0, row.duv_3_1],
    ]).unsqueeze(0)

    # h = four_point_homography_to_matrix(image_edges(img), -duv)
    # wrp_est = warp_perspective(img, h, dsize=img.shape[-2:])
    # wrp = (tensor_to_image(wrp, keepdim=False)*255.).astype(np.uint8)
    # wrp_est = (tensor_to_image(wrp_est, keepdim=False)*255.).astype(np.uint8)
    # concat = np.concatenate([wrp, wrp_est], axis=1)
    
    blend = create_blend_from_four_point_homography(wrp, img, -duv)
    blend = resize(blend, display_shape)
    img = resize(img, display_shape)
    blend = (tensor_to_image(blend, keepdim=False)*255.).astype(np.uint8)
    img = (tensor_to_image(img, keepdim=False)*255.).astype(np.uint8)
    concat = np.concatenate([blend, img[...,::-1]], axis=1)

    # display(PIL.Image.fromarray(concat))
    # display(TextDisplayObject("AutoLaparo label: {}, refined label: {}".format(motion_dict[label_df.iloc[row.vid]["Label"]], row.classes)).data)
    # clear_output(wait=True)

    text = "AutoLaparo label: {}, our re-label: {}".format(motion_dict[label_df.iloc[row.vid]["Label"]], row.classes)
    concat = cv2.putText(concat.copy(), text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    out.write(concat)
    if cnt >1000:
        break
    cnt+=1
out.release()